In [100]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
def getSteelList():
    url = "https://histock.tw/twclass/A021"
    user_agent = UserAgent()
    headers = {"user-agent": user_agent.random}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'lxml')

    a = soup.find_all("span", {"class":"w60 lft-p stockno"})
    list_ = []
    for i, test in enumerate(a):
        if i == 0:
            continue
        list_.append(test.text)
    return list_
list_ = getSteelList()
list_

['2002',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2012',
 '2013',
 '2014',
 '2015',
 '2017',
 '2020',
 '2022',
 '2023',
 '2024',
 '2025',
 '2027',
 '2028',
 '2029',
 '2030',
 '2031',
 '2032',
 '2033',
 '2034',
 '2038',
 '2069',
 '2211',
 '3004',
 '5007',
 '5538',
 '9958']

In [122]:
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
for year in [109]:
    service = Service('')  # Replace 'path_to_chromedriver' with the actual path to ChromeDriver
    driver = webdriver.Chrome(service=service)
    url = "https://mops.twse.com.tw/mops/web/stapap1"
    driver.get(url)
    time.sleep(3)
    dropdown = Select(driver.find_element(By.ID, "isnew"))
    dropdown.select_by_visible_text("歷史資料")
    for ticker in list_[12:]:
        time.sleep(1)
        
        if year != 113:
            loop_month = 13
        else:
            loop_month = 5
        final_df = pd.DataFrame()
        for month in range(1, loop_month):
            time.sleep(1)
            search_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='text'].textbox[name='co_id']"))
            )
            search_box.clear()
            search_box.send_keys(ticker)

            year_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='text'].textbox[name='year']"))
            )
            year_box.clear()
            year_box.send_keys(year)    

            dropdown = Select(driver.find_element(By.ID, "month"))
            dropdown.select_by_visible_text(str(month))
            search_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//input[@value=' 查詢 ']"))
            )
            search_button.click()

            time.sleep(3)

            WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CLASS_NAME, "hasBorder"))
            )

            pledge_table = driver.find_element(By.CLASS_NAME, "hasBorder")
            rows = pledge_table.find_elements(By.TAG_NAME, "tr")
            pledge_info = []
            for row in rows:
                cols = row.find_elements(By.TAG_NAME, "td")
                cols = [ele.text.strip() for ele in cols]
                if len(cols) == 9:  # Drop sublists with length not equal to 9
                    pledge_info.append(cols)
            pledge_df = pd.DataFrame(pledge_info)
            pledge_df['year']    = year
            pledge_df['month']   = month
            pledge_df['ticker']  = ticker

            final_df = pd.concat([final_df, pledge_df])
        print(ticker, year)
        final_df.columns = ['職稱', "姓名", "選任時持股", "目前持股", "設質股數", "	設質股數佔持股比例", "配偶、未成年子女及利用他人名義持有部份-內部人關係人目前持股合計", "配偶、未成年子女及利用他人名義持有部份-設質股數", "配偶、未成年子女及利用他人名義持有部份-設質比例", "year", "month", "ticker"]

        final_df.to_csv(f"pledge_{year}_{ticker}.csv")
    driver.quit()
    time.sleep(3)

2022 109
2023 109
2024 109
2025 109
2027 109


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=125.0.6422.113)
Stacktrace:
0   chromedriver                        0x00000001092626c8 chromedriver + 6149832
1   chromedriver                        0x0000000109259cea chromedriver + 6114538
2   chromedriver                        0x0000000108ce6d5c chromedriver + 400732
3   chromedriver                        0x0000000108cbc294 chromedriver + 225940
4   chromedriver                        0x0000000108d5f6fd chromedriver + 894717
5   chromedriver                        0x0000000108d75a98 chromedriver + 985752
6   chromedriver                        0x0000000108d56a53 chromedriver + 858707
7   chromedriver                        0x0000000108d266d5 chromedriver + 661205
8   chromedriver                        0x0000000108d26f6e chromedriver + 663406
9   chromedriver                        0x0000000109223d00 chromedriver + 5893376
10  chromedriver   

In [119]:
import os
import pandas as pd

# Step 1: Get all CSV files in the current directory
csv_files = [file for file in os.listdir('.') if file.endswith('.csv')]

# Step 2: Read each CSV file into a dataframe and store in a list
dataframes = []
for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# Step 3: Concatenate all dataframes by columns
concatenated_df = pd.concat(dataframes, axis=0)

# Step 4: Save the concatenated dataframe to a new CSV file
concatenated_df.to_csv('output/pledge_110_113.csv', index=False)


In [120]:
s = pd.read_csv("output/pledge_110_113.csv")
len(s)

31822